In [14]:
# Import required libraries
import requests
import json
import time
from datetime import datetime
import pandas as pd
from typing import List, Dict
import statistics

In [ ]:
# Configuration
API_URL = "http://localhost:8000/estimate-weight"
MODEL_NAME = "claude-haiku-4-5"

# List of offer IDs to test
OFFER_IDS = [
624730890959
568953817440
621796987802
602766644649
818322307945
732252682536
601747573294
626589994862
828176815369
597042514466
635994130681
711710642712
633573097044
590409798220
603450020772
525006763799
520829764809
626622859354
623409844150
634123488276
579325147274
762709360445
581477064514
520431501518
640911335840
635094132637
657563785032
590228198711
635181687313
560116006891
730244388583
554992493166
630198409357
552155078917
612899823898
589587782649
773013554799
827887580684
828240822385
564151541433
641430607491
745554091928
773124674725
563864567047
642071961825
597339298377
592211052698
734140093224
811517155585
737582148019
626199904545
602485551584
773053192401
629609946248
672160611914
679126677885
660206728469
641303285523
818819451892
682741208700
563377715620
712258308646
816086840615
773348662420
536577090188
716252308319
814597802859
624595542131
770053827786
617806400536
634542629387
641971552897
741803799246
826569563017
547896237507
755271783502
719950498546
813922865294
817763111343
826249040805
625921399000
742841052358
653479347253
626239290860
600116413912
728995711439
614254651476
810369663388
664330627564
631169391173
737831129958
583174483152
1086665641
817675662797
575114027644
638554547463
602634316361
650748853250
626583450545
714638138260
]

In [16]:
def make_api_call(offer_id: str, model_name: str) -> Dict:
    """
    Make a single API call to the weight estimation endpoint.
    
    Args:
        offer_id: The offer ID to estimate weight for
        model_name: The model name to use for estimation
    
    Returns:
        Dictionary containing response data and metadata
    """
    payload = {
        "offer_id": offer_id,
        "model_name": model_name
    }
    
    start_time = time.time()
    
    try:
        response = requests.post(API_URL, json=payload, timeout=120)
        end_time = time.time()
        
        result = {
            "offer_id": offer_id,
            "status_code": response.status_code,
            "success": response.status_code == 200,
            "response_time_seconds": end_time - start_time,
            "timestamp": datetime.now().isoformat(),
        }
        
        if response.status_code == 200:
            result["data"] = response.json()
        else:
            result["error"] = response.text
            
    except Exception as e:
        end_time = time.time()
        result = {
            "offer_id": offer_id,
            "status_code": None,
            "success": False,
            "response_time_seconds": end_time - start_time,
            "timestamp": datetime.now().isoformat(),
            "error": str(e)
        }
    
    return result

In [17]:
# Execute batch API calls
print(f"Starting batch API test for {len(OFFER_IDS)} offer(s)...")
print(f"API Endpoint: {API_URL}")
print(f"Model: {MODEL_NAME}")
print("=" * 80)

results = []
batch_start_time = time.time()

for idx, offer_id in enumerate(OFFER_IDS, 1):
    print(f"\n[{idx}/{len(OFFER_IDS)}] Processing offer_id: {offer_id}")
    
    result = make_api_call(offer_id, MODEL_NAME)
    results.append(result)
    
    status = "✓ SUCCESS" if result["success"] else "✗ FAILED"
    print(f"    Status: {status}")
    print(f"    Response time: {result['response_time_seconds']:.2f}s")
    
    if result["success"]:
        data = result.get("data", {})
        if "estimated_weight" in data:
            print(f"    Estimated weight: {data['estimated_weight']}")
    else:
        print(f"    Error: {result.get('error', 'Unknown error')}")

batch_end_time = time.time()
total_time = batch_end_time - batch_start_time

print("\n" + "=" * 80)
print("Batch processing complete!")

Starting batch API test for 2 offer(s)...
API Endpoint: http://localhost:8000/estimate-weight
Model: claude-haiku-4-5

[1/2] Processing offer_id: 624730890959
    Status: ✓ SUCCESS
    Response time: 9.30s

[2/2] Processing offer_id: 568953817440
    Status: ✓ SUCCESS
    Response time: 1.83s

Batch processing complete!


In [18]:
# Calculate statistics
successful_calls = [r for r in results if r["success"]]
failed_calls = [r for r in results if not r["success"]]
response_times = [r["response_time_seconds"] for r in results]

stats = {
    "total_requests": len(results),
    "successful_requests": len(successful_calls),
    "failed_requests": len(failed_calls),
    "success_rate_percent": (len(successful_calls) / len(results) * 100) if results else 0,
    "total_time_seconds": total_time,
    "total_time_formatted": f"{int(total_time // 60)}m {int(total_time % 60)}s",
    "average_response_time_seconds": statistics.mean(response_times) if response_times else 0,
    "median_response_time_seconds": statistics.median(response_times) if response_times else 0,
    "min_response_time_seconds": min(response_times) if response_times else 0,
    "max_response_time_seconds": max(response_times) if response_times else 0,
    "stdev_response_time_seconds": statistics.stdev(response_times) if len(response_times) > 1 else 0,
    "requests_per_second": len(results) / total_time if total_time > 0 else 0
}

# Display statistics
print("\n📊 PERFORMANCE STATISTICS")
print("=" * 80)
print(f"Total Requests:           {stats['total_requests']}")
print(f"Successful:               {stats['successful_requests']} ({stats['success_rate_percent']:.1f}%)")
print(f"Failed:                   {stats['failed_requests']}")
print(f"\nTotal Time:               {stats['total_time_formatted']} ({stats['total_time_seconds']:.2f}s)")
print(f"Average Response Time:    {stats['average_response_time_seconds']:.2f}s")
print(f"Median Response Time:     {stats['median_response_time_seconds']:.2f}s")
print(f"Min Response Time:        {stats['min_response_time_seconds']:.2f}s")
print(f"Max Response Time:        {stats['max_response_time_seconds']:.2f}s")
print(f"Std Dev Response Time:    {stats['stdev_response_time_seconds']:.2f}s")
print(f"Throughput:               {stats['requests_per_second']:.2f} requests/second")
print("=" * 80)


📊 PERFORMANCE STATISTICS
Total Requests:           2
Successful:               2 (100.0%)
Failed:                   0

Total Time:               0m 11s (11.13s)
Average Response Time:    5.56s
Median Response Time:     5.56s
Min Response Time:        1.83s
Max Response Time:        9.30s
Std Dev Response Time:    5.29s
Throughput:               0.18 requests/second


In [19]:
# Create a summary DataFrame
summary_data = []
for r in results:
    row = {
        "offer_id": r["offer_id"],
        "success": r["success"],
        "status_code": r["status_code"],
        "response_time_s": round(r["response_time_seconds"], 2),
        "timestamp": r["timestamp"]
    }
    
    if r["success"] and "data" in r:
        data = r["data"]
        row["estimated_weight"] = data.get("estimated_weight")
        row["unit"] = data.get("unit")
    else:
        row["estimated_weight"] = None
        row["unit"] = None
        row["error"] = r.get("error", "")[:100]  # Truncate error message
    
    summary_data.append(row)

df_summary = pd.DataFrame(summary_data)
print("\n📋 RESULTS SUMMARY")
print("=" * 80)
print(df_summary.to_string(index=False))
print("=" * 80)


📋 RESULTS SUMMARY
    offer_id  success  status_code  response_time_s                  timestamp estimated_weight unit
624730890959     True          200             9.30 2026-01-08T18:02:48.749560             None None
568953817440     True          200             1.83 2026-01-08T18:02:50.575032             None None


In [20]:
# Save results to JSON file
output_filename = f"batch_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

output_data = {
    "metadata": {
        "api_url": API_URL,
        "model_name": MODEL_NAME,
        "execution_timestamp": datetime.now().isoformat(),
        "total_offers_processed": len(results)
    },
    "statistics": stats,
    "results": results
}

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"\n💾 Results saved to: {output_filename}")


💾 Results saved to: batch_results_20260108_180302.json


In [21]:
# Optional: Display detailed results for successful calls
print("\n📝 DETAILED SUCCESSFUL RESPONSES")
print("=" * 80)

for idx, result in enumerate(successful_calls, 1):
    print(f"\n[{idx}] Offer ID: {result['offer_id']}")
    print(f"Response Time: {result['response_time_seconds']:.2f}s")
    print(f"Data:")
    print(json.dumps(result.get('data', {}), indent=2))
    print("-" * 80)


📝 DETAILED SUCCESSFUL RESPONSES

[1] Offer ID: 624730890959
Response Time: 9.30s
Data:
{
  "success": true,
  "offer_id": "624730890959",
  "skus_were_identical": false,
  "estimated_weights": [
    {
      "skus": [
        {
          "skuId": "4423361457251",
          "length_cm": 27.28,
          "width_cm": 22.19,
          "height_cm": 6.42,
          "weight_g": 541.0
        },
        {
          "skuId": "4423361457252",
          "length_cm": 27.28,
          "width_cm": 22.19,
          "height_cm": 6.42,
          "weight_g": 548.0
        },
        {
          "skuId": "4423361457253",
          "length_cm": 27.28,
          "width_cm": 22.19,
          "height_cm": 6.42,
          "weight_g": 573.0
        },
        {
          "skuId": "4423361457254",
          "length_cm": 27.28,
          "width_cm": 22.19,
          "height_cm": 6.42,
          "weight_g": 596.0
        },
        {
          "skuId": "4423361457255",
          "length_cm": 27.28,
          "wid